In [37]:
import numpy as np
import pandas as pd
from datetime import datetime
import datadotworld as dw

In [38]:
# 0
        # Without authentication.
        # data = pd.read_csv("https://query.data.world/s/xiz7cr6gvmuegh4bcl4r2trdrz5hnp")
        # data.info()
query = dw.query('siyeh/austin-animal-center-live-data', 
                 'SELECT * FROM austin_animal_center_outcomes')
data = query.dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133527 entries, 0 to 133526
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   animal_id         133527 non-null  object        
 1   name              92947 non-null   object        
 2   datetime          133527 non-null  datetime64[ns]
 3   monthyear         133527 non-null  datetime64[ns]
 4   date_of_birth     133527 non-null  object        
 5   outcome_type      133502 non-null  object        
 6   outcome_subtype   61302 non-null   object        
 7   animal_type       133527 non-null  object        
 8   sex_upon_outcome  133527 non-null  object        
 9   age_upon_outcome  133527 non-null  object        
 10  breed             133527 non-null  object        
 11  color             133527 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.2+ MB


<b>REMOVE duplicate rows, FILL null values</b>

In [39]:
# 1
data.drop_duplicates(inplace=True)

In [40]:
# 2
data.drop("monthyear", axis=1, inplace=True)

In [42]:
# 3
data["name"].fillna("Unknown", inplace=True)
data["name"] = data["name"].str.replace("*", "", regex=False)
data.loc[data["name"]=="", "name"] = "Unknown"

In [43]:
# 4
data["outcome_type"].fillna("Unknown", inplace=True)

In [44]:
# 5
data["outcome_subtype"].fillna("No Subtype", inplace=True)

In [45]:
# 6
data["sex_upon_outcome"].fillna("Unknown", inplace=True)

In [47]:
# 7
data["date_of_birth"] = pd.to_datetime(data["date_of_birth"], format="%Y-%m-%d")

#data.loc[data["Age upon Outcome"].isnull()]
#data.loc[data["Animal ID"]=="A834932", "Age upon Outcome"] = "1 year"
#data.loc[data["Animal ID"]=="A839109", "Age upon Outcome"] = "4 years"
#data.loc[data["Animal ID"]=="A837996", "Age upon Outcome"] = "3 years"

<b>ADJUST & CREATE additional COLUMNS</b>

In [48]:
# 8
data.loc[data["animal_id"]==data["name"], "name"] = "Unknown"

In [49]:
# 9
data["out_time2"] = np.where(data.datetime.dt.hour < 12, "AM", "PM")
data["datetime"] = data["datetime"].astype("str")
data[["out_date", "out_time1"]] = data["datetime"].str.split(pat=" ", n=1, expand=True)

In [50]:
# 10
data["out_date"] = pd.to_datetime(data["out_date"], format="%Y-%m-%d")
data.drop("datetime", axis=1, inplace=True)

In [52]:
# 11
data.rename(columns={"outcome_type":"out_type",
                     "outcome_subtype":"out_subtype",
                     "sex_upon_outcome":"out_sex",
                     "age_upon_outcome":"out_age"}, inplace=True)

In [58]:
data[data["out_age"].isnull()]

,animal_id,name,date_of_birth,out_type,out_subtype,animal_type,out_sex,out_age,breed,color,out_time2,out_date,out_time1


In [59]:
# 12
col1 = data[data["out_age"].isnull()]["out_date"].copy()
col2 = data[data["out_age"].isnull()]["date_of_birth"].copy()
data.loc[data["out_age"].isnull(), "out_age"] = (col1 - col2).astype("str")

col1 = data[data["out_age"]=="NULL"]["out_date"].copy()
col2 = data[data["out_age"]=="NULL"]["date_of_birth"].copy()
data.loc[data["out_age"]=="NULL", "out_age"] = (col1 - col2).astype("str")

In [60]:
# 13
data[["auo1", "auo2"]] = data["out_age"].str.split(n=1, expand=True)
data["auo1"] = data["auo1"].astype("int")
data.loc[data["auo1"]<0, "auo1"] = 0

In [61]:
# 14
conditions = [(data["auo2"]=="day") | (data["auo2"]=="days"),
              (data["auo2"]=="week") | (data["auo2"]=="weeks"),
              (data["auo2"]=="month") | (data["auo2"]=="months"),
              (data["auo2"]=="year") | (data["auo2"]=="years"),]
values = [1/30, 1/(30/7), 1, 12]
data["auo3"] = np.select(conditions, values)

In [62]:
# 15
data["out_age2"] = data["auo1"] * data["auo3"]
data.drop(["auo1", "auo2", "auo3"], axis=1, inplace=True)

In [63]:
# 16
data.sort_values(by="out_date", inplace=True)

<b>RECHECK data</b>

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132410 entries, 104789 to 132425
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   animal_id      132410 non-null  object        
 1   name           132410 non-null  object        
 2   date_of_birth  132410 non-null  datetime64[ns]
 3   out_type       132410 non-null  object        
 4   out_subtype    132410 non-null  object        
 5   animal_type    132410 non-null  object        
 6   out_sex        132410 non-null  object        
 7   out_age        132410 non-null  object        
 8   breed          132410 non-null  object        
 9   color          132410 non-null  object        
 10  out_date       132410 non-null  datetime64[ns]
 11  out_time1      132410 non-null  object        
 12  out_time2      132410 non-null  object        
 13  out_age2       132410 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(11)
me

In [ ]:
data

,animal_id,name,date_of_birth,out_type,out_subtype,animal_type,out_sex,out_age,breed,color,out_date,out_time1,out_time2,out_age2
104789,A664260,Unknown,2013-09-10,Transfer,Partner,Cat,Intact Female,3 weeks,Domestic Shorthair Mix,Black/White,2013-10-01,01:00:00,PM,0.700000
108452,A664263,Unknown,2013-09-22,Transfer,Partner,Cat,Intact Male,1 weeks,Domestic Shorthair Mix,Orange Tabby,2013-10-01,12:58:00,PM,0.233333
67115,A664261,Unknown,2013-09-10,Transfer,Partner,Cat,Intact Female,3 weeks,Domestic Shorthair Mix,Tortie,2013-10-01,01:00:00,PM,0.700000
93236,A664225,Lou,2010-09-30,Return to Owner,No Subtype,Dog,Neutered Male,3 years,Shih Tzu Mix,Gray/White,2013-10-01,12:13:00,PM,36.000000
64607,A663935,Unknown,1999-09-26,Transfer,Partner,Dog,Intact Male,14 years,Chihuahua Shorthair Mix,Brown,2013-10-01,02:15:00,PM,168.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132399,A843059,Bear,2016-09-25,Rto-Adopt,No Subtype,Dog,Neutered Male,5 years,Great Pyrenees,White,2021-10-12,02:27:00,PM,60.000000
132400,A844032,Lilu,2019-10-11,Return to Owner,No Subtype,Dog,Intact Female,2 years,Pit Bull Mix,Brown,2021-10-12,02:26:00,PM,24.000000
132401,A842103,Lily,2021-07-29,Adoption,Foster,Cat,Spayed Female,2 months,Domestic Shorthair,Brown Tabby,2021-10-12,03:46:00,PM,2.000000
132391,A842468,Spooky,2021-08-09,Adoption,Foster,Cat,Neutered Male,2 months,Domestic Shorthair,Black Smoke,2021-10-12,12:40:00,PM,2.000000


<b>EXPORT data</b>

In [ ]:
# 17
data.to_csv("preprocessed_AnimalCenter-AustinTX_Outcomes.csv", index=False)

In [ ]:
x = pd.read_csv("preprocessed_AnimalCenter-AustinTX_Outcomes.csv")
x.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132410 entries, 0 to 132409
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   animal_id      132410 non-null  object 
 1   name           132410 non-null  object 
 2   date_of_birth  132410 non-null  object 
 3   out_type       132410 non-null  object 
 4   out_subtype    132410 non-null  object 
 5   animal_type    132410 non-null  object 
 6   out_sex        132410 non-null  object 
 7   out_age        132410 non-null  object 
 8   breed          132410 non-null  object 
 9   color          132410 non-null  object 
 10  out_date       132410 non-null  object 
 11  out_time1      132410 non-null  object 
 12  out_time2      132410 non-null  object 
 13  out_age2       132410 non-null  float64
dtypes: float64(1), object(13)
memory usage: 14.1+ MB
